In [1]:
#Authors: Adam Lewandowski, Ivan Sladkov, Patrick English
import numpy as np
import tensorflow as tf
import pandas as pd
import string
import nltk
import os

#from spellchecker import SpellChecker
from sklearn.feature_extraction.text import TfidfVectorizer
from nltk.tokenize import TweetTokenizer
from nltk.stem.wordnet import WordNetLemmatizer
from sklearn.feature_extraction.text import HashingVectorizer

In [4]:
raw_tweets_df = pd.read_csv("data/training.1600000.processed.noemoticon.csv", encoding = "ISO-8859-1", header=None)
raw_tweets_df

FileNotFoundError: [Errno 2] No such file or directory: 'data/training.1600000.processed.noemoticon.csv'

In [8]:
print(raw_tweets_df[1])
#Baseline processing
tknzr = TweetTokenizer()

#This can take a while
#tokenized_tweets = raw_tweets_df[5].copy().apply(lambda x : x.lower())

# If you don't want to wait
tokenized_tweets = raw_tweets_df[5][:3200].copy().apply(lambda x : x.lower().encode('ascii', 'ignore'))

tokenized_tweets = tokenized_tweets.apply(lambda x : tknzr.tokenize(x))

0          1467810369
1          1467810672
2          1467810917
3          1467811184
4          1467811193
              ...    
1599995    2193601966
1599996    2193601969
1599997    2193601991
1599998    2193602064
1599999    2193602129
Name: 1, Length: 1600000, dtype: int64


In [73]:
#Should things like ... stay? 
def remove_punctuation(tweets):
    t = tweets.copy()
    def remove_punct(tokens):
        if '.' in tokens:
            tokens.remove('.')
        if '?' in tokens:
            tokens.remove('?')
        if '!' in tokens:
            tokens.remove('!')
        if ',' in tokens:
            tokens.remove(',')
        if '\'' in tokens:
            tokens.remove('\'')
        if '"' in tokens:
            tokens.remove('"')
        if '#' in tokens:
            tokens.remove('#')
        if '' in tokens:
            tokens.remove('#')
            
        return tokens
    return t.apply(lambda tweet : remove_punct(tweet))

def remove_stop_words(tweets):
    t = tweets.copy()
    from nltk.corpus import stopwords
    sw = set(stopwords.words('english')) 
    return t.apply(lambda tweet:[w for w in tweet if not w in sw])

#TODO add spell correction
def correct_spelling(tweets):
    return tweets

def pos_tag(tweets):
    t = tweets.copy()
    return t.apply(lambda tweet: nltk.pos_tag(tweet))

#TBD
def emoticon_transcoder(tweets):
    return tweets

def remove_pos_tags(tweets):
    t = tweets.copy()
    return t.apply(lambda tweet: [w[0] for w in tweet])

def identity_tokenizer(text):
    return text

def scikit_vectorize(tweets, min_freq):
    t = tweets.copy()
    vectorizer = TfidfVectorizer(lowercase=False, tokenizer=identity_tokenizer, min_df=min_freq, dtype=np.float32)
    vectors = vectorizer.fit_transform(t)
    feature_names = vectorizer.get_feature_names()
    dense = vectors.todense()
    denselist = dense.tolist()
    return pd.DataFrame(denselist, columns=feature_names)

def scikit_hash_vectorize(tweets):
    t = tweets.copy()
    vectorizer = HashingVectorizer(lowercase=False, n_features=2**8, tokenizer=identity_tokenizer, dtype=np.float32)
    vectors = vectorizer.fit_transform(t)
    dense = vectors.todense()
    denselist = dense.tolist()
    return pd.DataFrame(denselist)

def str_list_to_list(tweets):
    t = tweets.copy()
    import ast 
    t["tweets"] = t["tweets"].apply(lambda x: ast.literal_eval(x))
    return t

def lemmatize(tweets):
    t = tweets.copy()
    
    def get_wordnet_pos(treebank_tag):
        from nltk.corpus import wordnet
        if treebank_tag.startswith('J'):
            return wordnet.ADJ
        elif treebank_tag.startswith('V'):
            return wordnet.VERB
        elif treebank_tag.startswith('N'):
            return wordnet.NOUN
        elif treebank_tag.startswith('R'):
            return wordnet.ADV
        else:
            return wordnet.NOUN
    lem = WordNetLemmatizer()
    def lemmatize_word(w):
        transcoded_tag = get_wordnet_pos(w[1])
        wl = lem.lemmatize(w[0], transcoded_tag)
        return (wl,w[1])
    return t.apply(lambda tweet:[lemmatize_word(w) for w in tweet])

def remove_punct(tokens):
        if '.' in tokens:
            tokens.remove('.')
        if '?' in tokens:
            tokens.remove('?')
        if '!' in tokens:
            tokens.remove('!')
        if ',' in tokens:
            tokens.remove(',')
        return tokens
    
def remove_one_char_trash(tweets):
    return tweets.apply(lambda tweet: [s for s in tweet if len(s) >= 2])

arr = ["gdfgfd", "]","rer"]
arr = [s for s in arr if len(s) >= 2]
print(arr)

['gdfgfd', 'rer']


In [6]:
#Pre-processing
#comment lines to skip some steps
#You need to run things in this order
tweets = remove_punctuation(tokenized_tweets)
tweets = remove_one_char_trash(tweets)
tweets = remove_stop_words(tweets)
tweets = pos_tag(tweets)
tweets = lemmatize(tweets)
tweets = remove_pos_tags(tweets)
#tweets = merge_tokens(tweets)

In [71]:
tw = raw_tweets_df.drop(columns=[1,2,3,4,5])
tw["target"] =tw[0]
tw = tw.drop(columns=[0])
tw["tweets"] = tweets
print(tw)
#print(tweets)

         target                                             tweets
0             0  [@switchfoot, http://twitpic.com/2y1zl, awww, ...
1             0  [upset, can't, update, facebook, texting, ...,...
2             0  [@kenichan, dive, many, time, ball, manage, sa...
3             0             [whole, body, feel, itchy, like, fire]
4             0   [@nationwideclass, behave, i'm, mad, can't, see]
...         ...                                                ...
1599995       4                [woke, school, best, feeling, ever]
1599996       4  [thewdb.com, cool, hear, old, walt, interview,...
1599997       4               [ready, mojo, makeover, ask, detail]
1599998       4  [happy, 38th, birthday, boo, alll, time, tupac...
1599999       4  [happy, #charitytuesday, @thenspcc, @sparkscha...

[1600000 rows x 2 columns]


In [74]:

import os
if os.path.exists("processed_tweets.csv"):
    os.remove("processed_tweets.csv")

tw.to_csv("processed_tweets.csv", encoding = "ISO-8859-1", index=False)


In [75]:
tweets = pd.read_csv("processed_tweets.csv", encoding = "ISO-8859-1")
tweets = str_list_to_list(tweets)

In [76]:
print(tweets)
#tweetsVector = scikit_vectorize(tweets, 2)

#print(tweets.array)
#tweetsVector
#print(tweetsVector)

         target                                             tweets
0             0  [@switchfoot, http://twitpic.com/2y1zl, awww, ...
1             0  [upset, can't, update, facebook, texting, ...,...
2             0  [@kenichan, dive, many, time, ball, manage, sa...
3             0             [whole, body, feel, itchy, like, fire]
4             0   [@nationwideclass, behave, i'm, mad, can't, see]
...         ...                                                ...
1599995       4                [woke, school, best, feeling, ever]
1599996       4  [thewdb.com, cool, hear, old, walt, interview,...
1599997       4               [ready, mojo, makeover, ask, detail]
1599998       4  [happy, 38th, birthday, boo, alll, time, tupac...
1599999       4  [happy, #charitytuesday, @thenspcc, @sparkscha...

[1600000 rows x 2 columns]


In [57]:
tweetsVector = scikit_hash_vectorize(tweets)

/usr/lib/python3.8/site-packages/sklearn/feature_extraction/text.py:507: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn("The parameter 'token_pattern' will not be used"


In [61]:
if os.path.exists("vec.csv"):
    os.remove("vec.csv")

tweetsVector.to_csv("vec.csv", encoding = "ISO-8859-1", index=False)

In [1]:
print(len(tweetsVector.iloc[0]))
#print(tweetsVector.keys())

NameError: name 'tweetsVector' is not defined

In [58]:
print(tweetsVector)

         0    1    2    3    4         5         6    7    8    9    ...  246  \
0        0.0  0.0  0.0  0.0  0.0  0.288675  0.000000  0.0  0.0  0.0  ...  0.0   
1        0.0  0.0  0.0  0.0  0.0  0.000000  0.000000  0.0  0.0  0.0  ...  0.0   
2        0.0  0.0  0.0  0.0  0.0  0.000000  0.000000  0.0  0.0  0.0  ...  0.0   
3        0.0  0.0  0.0  0.0  0.0  0.000000  0.000000  0.0  0.0  0.0  ...  0.0   
4        0.0  0.0  0.0  0.0  0.0  0.000000  0.000000  0.0  0.0  0.0  ...  0.0   
...      ...  ...  ...  ...  ...       ...       ...  ...  ...  ...  ...  ...   
1599995  0.0  0.0  0.0  0.0  0.0  0.000000  0.000000  0.0  0.0  0.0  ...  0.0   
1599996  0.0  0.0  0.0  0.0  0.0  0.000000 -0.377964  0.0  0.0  0.0  ...  0.0   
1599997  0.0  0.0  0.0  0.0  0.0  0.000000  0.000000  0.0  0.0  0.0  ...  0.0   
1599998  0.0  0.0  0.0  0.0  0.0  0.000000  0.000000  0.0  0.0  0.0  ...  0.0   
1599999  0.0  0.0  0.0  0.0  0.0  0.000000  0.000000  0.0  0.0  0.0  ...  0.0   

         247       248     